# 2002 年-2018 年上海机动车拍照拍卖

(1) 哪一次拍卖的中标率首次小于5%？  
(2) 按年统计拍卖最低价的下列统计量：最大值、均值、0.75 分位数，要求
显示在同一张表上。  
(3) 将第一列时间列拆分成两个列，一列为年份（格式为20××），另一列为
月份（英语缩写），添加到列表作为第一第二列，并将原表第一列删除，
其他列依次向后顺延。  
(4) 现在将表格行索引设为多级索引，外层为年份，内层为原表格第二至第
五列的变量名，列索引为月份。  
(5) 一般而言某个月最低价与上月最低价的差额，会与该月均值与上月均值
的差额具有相同的正负号，哪些拍卖时间不具有这个特点？  
(6) 将某一个月牌照发行量与其前两个月发行量均值的差额定义为发行增
益，最初的两个月用0 填充，求发行增益极值出现的时间。

In [59]:
import numpy as np
import pandas as pd

df = pd.read_csv('2002年-2018年上海机动车拍照拍卖.csv')
df.head()

Date  Total number of license issued  lowest price   avg price  \
0  2-Jan                            1400          13600      14735   
1  2-Feb                            1800          13100      14057   
2  2-Mar                            2000          14300      14662   
3  2-Apr                            2300          16000      16334   
4  2-May                            2350          17800      18357   

   Total number of applicants  
0                        3718  
1                        4590  
2                        5190  
3                        4806  
4                        4665

## A1

In [60]:
temp1 = df.copy()
temp1['percent'] = temp1['Total number of license issued'] / \
                temp1['Total number of applicants']
temp1.loc[temp1['percent'] < 0.05].head(1)

Date  Total number of license issued  lowest price   avg price  \
159  15-May                            7482          79000      79099   

     Total number of applicants   percent  
159                      156007  0.047959

## A3

In [61]:
temp3 = df.copy()
cols = temp3['Date'].apply(lambda x: x.split('-'))
cols = np.array(cols.tolist())
temp3['year'] = cols[:, 0].astype(np.int) + 2000
temp3['month'] = cols[:, 1]
temp3.drop('Date', axis = 1, inplace = True)
columns = ['year', 'month', 'Total number of license issued', \
           'lowest price ', 'avg price', 'Total number of applicants']
temp3 = temp3.reindex(columns = columns)
temp3.head()

year month  Total number of license issued  lowest price   avg price  \
0  2002   Jan                            1400          13600      14735   
1  2002   Feb                            1800          13100      14057   
2  2002   Mar                            2000          14300      14662   
3  2002   Apr                            2300          16000      16334   
4  2002   May                            2350          17800      18357   

   Total number of applicants  
0                        3718  
1                        4590  
2                        5190  
3                        4806  
4                        4665

## A2

In [62]:
temp2 = temp3.copy()
def percetile_75(x):
    return np.percentile(x, 0.75)
temp2.loc[:, ['year', 'lowest price ']].groupby('year').agg(['max', 'mean', \
                                                             percetile_75])

lowest price                            
               max          mean percetile_75
year                                         
2002         30800  20316.666667     13141.25
2003         38500  31983.333333     19212.50
2004         44200  29408.333333     11377.50
2005         37900  31908.333333     25016.50
2006         39900  37058.333333     27502.25
2007         53800  45691.666667     38549.50
2008         37300  29945.454545      9127.50
2009         36900  31333.333333     26649.50
2010         44900  38008.333333     12660.50
2011         53800  47958.333333     38786.75
2012         68900  61108.333333     53014.50
2013         90800  79125.000000     73507.25
2014         74600  73816.666667     73224.75
2015         85300  80575.000000     74049.50
2016         88600  85733.333333     82274.25
2017         93500  90616.666667     87616.50
2018         89000  87825.000000     86933.00

## A4

In [136]:
temp4 = temp3.copy()
temp4 = pd.pivot_table(temp4, index = ['year'], 
                      columns = 'month').stack(0)
columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct',
          'Nov', 'Dec']
temp4 = temp4.reindex(columns = columns)
temp4.head()

month                                    Jan      Feb      Mar      Apr  \
year                                                                      
2002 Total number of applicants       3718.0   4590.0   5190.0   4806.0   
     Total number of license issued   1400.0   1800.0   2000.0   2300.0   
     avg price                       14735.0  14057.0  14662.0  16334.0   
     lowest price                    13600.0  13100.0  14300.0  16000.0   
2003 Total number of applicants       9442.0  12030.0  11219.0   8794.0   

month                                    May      Jun      Jul      Aug  \
year                                                                      
2002 Total number of applicants       4665.0   4502.0   3774.0   4640.0   
     Total number of license issued   2350.0   2800.0   3000.0   3000.0   
     avg price                       18357.0  20178.0  20904.0  21601.0   
     lowest price                    17800.0  19600.0  19800.0  21000.0   
2003 Total number of applicants      14634.0  15507.0  11929.0   9315.0   

month                                    Sep      Oct      Nov      Dec  
year                                                                     
2002 Total number of applicants       4393.0   4661.0   4021.0   3525.0  
     Total number of license issued   3200.0   3200.0   3200.0   3600.0  
     avg price                       24040.0  27040.0  31721.0  27848.0  
     lowest price                    23600.0  26400.0  30800.0  27800.0  
2003 Total number of applicants       8532.0   9383.0   9849.0  10491.0

## A5

In [74]:
temp5 = temp3.copy()
temp = temp5.drop(index = [0])
index_list = []
for i in range(1, temp5.shape[0]):
    f1 = temp['lowest price '][i] - temp5['lowest price '][i - 1]
    f2 = temp['avg price'][i] - temp5['avg price'][i - 1]
    if f1 * f2 < 0:
        index_list.append(i)
temp5.iloc[index_list, 0: 2]

year month
21   2003   Oct
22   2003   Nov
29   2004   Jun
36   2005   Jan
37   2005   Feb
44   2005   Sep
52   2006   May
56   2006   Sep
60   2007   Jan
61   2007   Feb
71   2007   Dec
128  2012   Oct

## A6

In [97]:
temp6 = temp3.copy()
array = temp6['Total number of license issued'].values
for i in range(array.size - 1, 1, -1):
    array[i] -= (array[i - 1] + array[i - 2]) / 2
array[0: 2] = 0
temp6['A6'] = array
display(temp6[temp6['A6'] == temp6['A6'].max()].loc[:, ['year', 'month']])
display(temp6[temp6['A6'] == temp6['A6'].min()].loc[:, ['year', 'month']])

year month
72  2008   Jan

year month
74  2008   Apr